In [88]:
!pip install numpy pandas requests tqdm rectools

In [78]:
from pprint import pprint

import numpy as np
import pandas as pd

import requests
from tqdm.auto import tqdm

from rectools import Columns
from rectools.dataset import Dataset,Interactions
from rectools.models import ImplicitItemKNNWrapperModel

In [ ]:
url = 'https://github.com/irsafilo/KION_DATASET/raw/f69775be31fa5779907cf0a92ddedb70037fb5ae/data_original.zip'

In [ ]:
req = requests.get(url, stream=True)

with open('kion.zip', 'wb') as fd:
    total_size_in_bytes = int(req.headers.get('Content-Length', 0))
    progress_bar = tqdm(desc='kion dataset download', total=total_size_in_bytes, unit='iB', unit_scale=True)
    for chunk in req.iter_content(chunk_size=2 ** 20):
        progress_bar.update(len(chunk))
        fd.write(chunk)

kion dataset download:   0%|          | 0.00/78.8M [00:00<?, ?iB/s]

In [ ]:
import zipfile as zf

files = zf.ZipFile('kion.zip','r')
files.extractall()
files.close()

In [65]:
interactions = pd.read_csv('data_original/interactions.csv', parse_dates=["last_watch_dt"])

interactions.rename(
    columns={
        'last_watch_dt': Columns.Datetime,
        'total_dur': Columns.Weight
    },
    inplace=True)
interactions.head()

,user_id,item_id,datetime,weight,watched_pct
0,176549,9506,2021-05-11,4250,72.0
1,699317,1659,2021-05-29,8317,100.0
2,656683,7107,2021-05-09,10,0.0
3,864613,7638,2021-07-05,14483,100.0
4,964868,9506,2021-04-30,6725,100.0


In [66]:
interactions = Interactions(interactions)

interactions.df.head()

,user_id,item_id,datetime,weight,watched_pct
0,176549,9506,2021-05-11,4250.0,72.0
1,699317,1659,2021-05-29,8317.0,100.0
2,656683,7107,2021-05-09,10.0,0.0
3,864613,7638,2021-07-05,14483.0,100.0
4,964868,9506,2021-04-30,6725.0,100.0


In [67]:
users = pd.read_csv('data_original/users.csv')
items = pd.read_csv('data_original/items.csv')

In [68]:
users.head()


,user_id,age,income,sex,kids_flg
0,973171,age_25_34,income_60_90,М,1
1,962099,age_18_24,income_20_40,М,0
2,1047345,age_45_54,income_40_60,Ж,0
3,721985,age_45_54,income_20_40,Ж,0
4,704055,age_35_44,income_60_90,Ж,0


In [69]:
items.head()

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
0,10711,film,Поговори с ней,Hable con ella,2002.0,"драмы, зарубежные, детективы, мелодрамы",Испания,NaN,16.0,NaN,Педро Альмодовар,"Адольфо Фернандес, Ана Фернандес, Дарио Гранди...",Мелодрама легендарного Педро Альмодовара «Пого...,"Поговори, ней, 2002, Испания, друзья, любовь, ..."
1,2508,film,Голые перцы,Search Party,2014.0,"зарубежные, приключения, комедии",США,NaN,16.0,NaN,Скот Армстронг,"Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон ...",Уморительная современная комедия на популярную...,"Голые, перцы, 2014, США, друзья, свадьбы, прео..."
2,10716,film,Тактическая сила,Tactical Force,2011.0,"криминал, зарубежные, триллеры, боевики, комедии",Канада,NaN,16.0,NaN,Адам П. Калтраро,"Адриан Холмс, Даррен Шалави, Джерри Вассерман,...",Профессиональный рестлер Стив Остин («Все или ...,"Тактическая, сила, 2011, Канада, бандиты, ганг..."
3,7868,film,45 лет,45 Years,2015.0,"драмы, зарубежные, мелодрамы",Великобритания,NaN,16.0,NaN,Эндрю Хэй,"Александра Риддлстон-Барретт, Джеральдин Джейм...","Шарлотта Рэмплинг, Том Кортни, Джеральдин Джей...","45, лет, 2015, Великобритания, брак, жизнь, лю..."
4,16268,film,Все решает мгновение,NaN,1978.0,"драмы, спорт, советские, мелодрамы",СССР,NaN,12.0,Ленфильм,Виктор Садовский,"Александр Абдулов, Александр Демьяненко, Алекс...",Расчетливая чаровница из советского кинохита «...,"Все, решает, мгновение, 1978, СССР, сильные, ж..."


In [72]:
def headtail(df):
    return pd.concat([df.df.head(), df.df.tail()])

headtail(interactions)

,user_id,item_id,datetime,weight,watched_pct
0,176549,9506,2021-05-11,4250.0,72.0
1,699317,1659,2021-05-29,8317.0,100.0
2,656683,7107,2021-05-09,10.0,0.0
3,864613,7638,2021-07-05,14483.0,100.0
4,964868,9506,2021-04-30,6725.0,100.0
5476246,648596,12225,2021-08-13,76.0,0.0
5476247,546862,9673,2021-04-13,2308.0,49.0
5476248,697262,15297,2021-08-20,18307.0,63.0
5476249,384202,16197,2021-04-19,6203.0,100.0
5476250,319709,4436,2021-08-15,3921.0,45.0


In [74]:
interactions.df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5476251 entries, 0 to 5476250
Data columns (total 5 columns):
 #   Column       Dtype         
---  ------       -----         
 0   user_id      int64         
 1   item_id      int64         
 2   datetime     datetime64[ns]
 3   weight       float64       
 4   watched_pct  float64       
dtypes: datetime64[ns](1), float64(2), int64(2)
memory usage: 208.9 MB


In [ ]:
sample_users = [57607, 403227, 70720]
df = interactions[interactions[Columns.User].isin(sample_users)].sort_values("user_id").reset_index(drop=True)
del df[Columns.Datetime], df[Columns.Weight], df['watched_pct']
df

,user_id,item_id,is_train
0,57607,4151,True
1,57607,10440,True
2,57607,13865,False
3,70720,4880,True
4,70720,4881,True
5,70720,6327,True
6,403227,6353,True
7,403227,1736,True
8,403227,5336,False
9,403227,181,True


In [83]:
import time
from copy import deepcopy
from rectools.models.popular import PopularModel
from rectools.models.random import RandomModel
from rectools.model_selection import TimeRangeSplitter
from rectools.metrics import Precision, Recall, MAP, MeanInvUserFreq, Serendipity, calc_metrics
import rectools
from sklearn.model_selection import RepeatedStratifiedKFold

In [86]:
# Define models
popular_model = PopularModel()
random_model = RandomModel(random_state=32)

models = {
    'popular': popular_model,
    'random': random_model
}

# Define metrics
metrics = {
    'Precision@1': Precision(k=1),
    'Precision@5': Precision(k=5),
    "Precision@10": Precision(k=10),
    'Recall@5': Recall(k=5),
    'MAP@5': MAP(k=5),
    "novelty": MeanInvUserFreq(k=10),
    "serendipity": Serendipity(k=10),
}

K_RECOS = 10




In [87]:
from tqdm.auto import tqdm
results = []

n_splits = 3

cv = TimeRangeSplitter(
    test_size="14D",
    n_splits=n_splits,
    filter_already_seen=True,
    filter_cold_items=True,
    filter_cold_users=True,
)

fold_iterator = cv.split(interactions, collect_fold_stats=True)




for train_ids, test_ids, fold_info in tqdm((fold_iterator), total=n_splits):
  print(f"\n==================== Fold {fold_info['i_split']}")
  pprint(fold_info)

  df_train = interactions.df.iloc[train_ids]
  dataset = Dataset.construct(df_train)

  df_test = interactions.df.iloc[test_ids][Columns.UserItem]
  test_users = np.unique(df_test[Columns.User])

  # Catalog is set of items that we recommend.
  # Sometimes we recommend not all items from train.
  catalog = df_train[Columns.Item].unique()

  for model_name, model in models.items():
      model.fit(dataset)
      recos = model.recommend(
            users=test_users,
            dataset=dataset,
            k=K_RECOS,
            filter_viewed=True,
        )
      metric_values = calc_metrics(
            metrics,
            reco=recos,
            interactions=df_test,
            prev_interactions=df_train,
            catalog=catalog,
        )
      res = {"fold": fold_info["i_split"], "model": model_name}
      res.update(metric_values)
      results.append(res)

  0%|          | 0/3 [00:00<?, ?it/s]


==================== Fold 0
{'end': Timestamp('2021-07-26 00:00:00', freq='14D'),
 'i_split': 0,
 'start': Timestamp('2021-07-12 00:00:00', freq='14D'),
 'test': 398993,
 'test_items': 7394,
 'test_users': 122488,
 'train': 3239125,
 'train_items': 14730,
 'train_users': 646423}

==================== Fold 1
{'end': Timestamp('2021-08-09 00:00:00', freq='14D'),
 'i_split': 1,
 'start': Timestamp('2021-07-26 00:00:00', freq='14D'),
 'test': 458757,
 'test_items': 7711,
 'test_users': 135624,
 'train': 3892558,
 'train_items': 15085,
 'train_users': 742256}

==================== Fold 2
{'end': Timestamp('2021-08-23 00:00:00', freq='14D'),
 'i_split': 2,
 'start': Timestamp('2021-08-09 00:00:00', freq='14D'),
 'test': 521381,
 'test_items': 7705,
 'test_users': 151629,
 'train': 4649162,
 'train_items': 15415,
 'train_users': 850489}


In [88]:
# Aggregate metrics by folds and compare models
pivot_results = pd.DataFrame(results).drop(columns="fold").groupby(["model"], sort=False).agg(["mean", "std"])
mean_metric_subset = [(metric, agg) for metric, agg in pivot_results.columns if agg == 'mean']
(
  pivot_results.style
  .highlight_min(subset=mean_metric_subset, color='lightcoral', axis=0)
  .highlight_max(subset=mean_metric_subset, color='lightgreen', axis=0)
)